Cats and Dogs Classifier
================================================================


In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
tf.__version__

'2.7.0'

### Preprocessing


Training Set Augmentation

In [3]:
train_augment = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 90,
    shear_range = 0.2,
    width_shift_range=0.5, 
    height_shift_range=0.5,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip= True,
    preprocessing_function= preprocess_input

)

training_set = train_augment.flow_from_directory('dataset/training_set',
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'binary')


Found 8000 images belonging to 2 classes.


Test Set Rescaling

In [4]:
test_rescale = ImageDataGenerator(
    rescale = 1./255,
    preprocessing_function= preprocess_input

    )

test_set = test_rescale.flow_from_directory('dataset/test_set',
                                            target_size = (150, 150),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


### Building the CNN

Load and Freeze VGG-16 

In [5]:
from tensorflow.keras.applications.vgg16 import VGG16

vgg = VGG16(
    weights='imagenet', 
    include_top = False, 
    input_shape=(150,150,3)
    )
# Freeze VGG-16
vgg.trainable = False



2021-12-31 08:28:58.010933: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-31 08:28:58.011081: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



Create model on top 

In [6]:
#init
model  = tf.keras.models.Sequential()
#input
model.add(tf.keras.layers.Flatten())
#connected layers
model.add(tf.keras.layers.Dense(units=128, activation = 'relu'))
model.add(tf.keras.layers.Dense(units=24, activation = 'relu'))
#output layers
model.add(tf.keras.layers.Dense(units=1, activation = 'softmax'))

Compile and Fit Model

In [7]:
from tensorflow.keras.callbacks import EarlyStopping


#compile model
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

#early stopping so model doesn't overfit
e = EarlyStopping(monitor='val_loss', mode='min', patience=5, restore_best_weights=True)


#fit model
model.fit(
    batch_size = 32, 
    x = training_set, 
    validation_data=test_set,
    epochs=50, 
    callbacks=e)

2021-12-31 08:29:07.858312: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-31 08:29:08.038481: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50
250/250 [==============================] - ETA: 0s - loss: 0.7756 - accuracy: 0.5000

2021-12-31 08:29:38.553757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 34s 134ms/step - loss: 0.7756 - accuracy: 0.5000 - val_loss: 0.7465 - val_accuracy: 0.5000
Epoch 2/50
250/250 [==============================] - 33s 130ms/step - loss: 0.7450 - accuracy: 0.5000 - val_loss: 0.7183 - val_accuracy: 0.5000
Epoch 3/50
250/250 [==============================] - 33s 131ms/step - loss: 0.7246 - accuracy: 0.5000 - val_loss: 0.7323 - val_accuracy: 0.5000
Epoch 4/50
250/250 [==============================] - 33s 132ms/step - loss: 0.7159 - accuracy: 0.5000 - val_loss: 0.7287 - val_accuracy: 0.5000
Epoch 5/50
127/250 [==============>...............] - ETA: 14s - loss: 0.7068 - accuracy: 0.4948

KeyboardInterrupt: 